In [3]:
import tensorflow as tf

In [4]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf.__version__

'1.12.0'

In [5]:
print(tf.get_default_graph())

In [11]:
my_graph = tf.Graph()
with my_graph.as_default():
    my_x = tf.constant(3)
    my_y = my_x ** 2

# create session declared
with tf.Session(graph=my_graph) as sess:
    out = sess.run([my_x, my_y])
    print(out)

[3, 9]


In [8]:
x = tf.constant(3)
y = x**2

sess = tf.Session()
print(sess.run(x))
print(sess.run(y))
sess.close()

3
9


In [12]:
# create default session
with tf.Session() as sess:
    out = sess.run([x, y])
    print(out)

[3, 9]


In [17]:
import pandas as pd
import numpy as np

In [18]:
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['const'] = np.ones(df.shape[0])
df.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,const
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,1.0
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,1.0
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,1.0
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,1.0
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88,1.0


In [20]:
n, m = df.shape

X = tf.placeholder(tf.float32, shape=(n, m))
y = tf.placeholder(tf.float32, shape=(n, 1))

XT = tf.transpose(X)
w = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
y_pred = tf.matmul(X, w)

with tf.Session() as sess:
    y_pred_ = sess.run(y_pred, \
                      feed_dict={X: df.values, y: boston.target.reshape(-1, 1)})

print("expected house price: ", y_pred_[19], "actual house price: ", boston.target[19])

expected house price:  [18.405354] actual house price:  18.2


In [21]:
from sklearn.datasets import load_iris

iris = load_iris()
idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2].astype('float32')
y_data = (iris.target[idx] - 1.0)[:, np.newaxis].astype('float32')

# optimization
- `get_variable`: 재사용
- `reduce_sum`: 설정 축의 요소 모두 더한 값 구함

In [30]:
tf.reset_default_graph() # del all graph, which have been made

np.random.seed(0)
with tf.variable_scope("perceptron", reuse=tf.AUTO_REUSE):
    w = 1e-3 * tf.get_variable("weight", [2, 1], dtype=tf.float32,
                              initializer=tf.random_uniform_initializer())
    b = 1e-3 * tf.get_variable("bias", [], dtype=tf.float32,
                              initializer=tf.random_uniform_initializer())
    z = tf.tanh(tf.matmul(X_data, w) + b)

zero = np.zeros(1, dtype=np.float32)[0]
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, z)))
optimizer = tf.train.GradientDescentOptimizer(1e-1)
train = optimizer.minimize(cost)



# create tensorboard 용도 log
- `summary.histogram`: 저장하려는 값이 다차원의 값

In [33]:
w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.scalar("b", b)
z_summary = tf.summary.histogram
cost_summary = tf.summary.scalar('cost', cost)
merged = tf.summary.merge_all()

init = tf.global_variables_initializer()
with tf.Session() as sess:
    writer = tf.summary.FileWriter("log", sess.graph)
    sess.run(init)
    for i in range(200):
        if i % 10 == 0:
            _, cost_, summary = sess.run([train, cost, merged])
            print("{:4d} : {:.5f}".format(i, cost_))
        else:
            _, summary = sess.run([train, merged])
        writer.add_summary(summary, i)
    y_pred = sess.run(tf.sign(z))

   0 : 0.24403
  10 : 0.14950
  20 : 0.05497
  30 : 0.00027
  40 : 0.00027
  50 : 0.00021
  60 : 0.00021
  70 : 0.00017
  80 : 0.00020
  90 : 0.00017
 100 : 0.00020
 110 : 0.00016
 120 : 0.00017
 130 : 0.00017
 140 : 0.00017
 150 : 0.00017
 160 : 0.00017
 170 : 0.00017
 180 : 0.00014
 190 : 0.00016
